In [31]:
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
import numpy as np
import pandas as pd


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
### DNN Regression
# It is now time to implements single-input and multiple-inputs DNN models
print('---------------------------------------------------------------------------------')
print('Initializing DNN program')
print('---------------------------------------------------------------------------------')
print('')
#---------------------------------------------------------------------
# Importing Libraries

print('---------------------------------------------------------------------------------')
print('Importing Libraries')
print('---------------------------------------------------------------------------------')
print('')

import time
import tensorflow as tf
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
#import keras
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout
#from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
#from keras.callbacks import EarlyStopping
#from keras.preprocessing import sequence
#from keras.utils import pad_sequences
#from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import shap
np.random.seed(0)

#---------------------------------------------------------------------
# Defining metric equations

print('---------------------------------------------------------------------------------')
print('Defining Metric Equations')
print('---------------------------------------------------------------------------------')
print('')

def ACC(TP,TN,FP,FN):
    Acc = (TP+TN)/(TP+FP+FN+TN)
    return Acc
def ACC_2 (TP, FN):
    ac = (TP/(TP+FN))
    return ac
def PRECISION(TP,FP):
    Precision = TP/(TP+FP)
    return Precision
def RECALL(TP,FN):
    Recall = TP/(TP+FN)
    return Recall
def F1(Recall, Precision):
    F1 = 2 * Recall * Precision / (Recall + Precision)
    return F1
def BACC(TP,TN,FP,FN):
    BACC =(TP/(TP+FN)+ TN/(TN+FP))*0.5
    return BACC
def MCC(TP,TN,FP,FN):
    MCC = (TN*TP-FN*FP)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**.5)
    return MCC
def AUC_ROC(y_test_bin,y_score):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    auc_avg = 0
    counting = 0
    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
     # plt.plot(fpr[i], tpr[i], color='darkorange', lw=2)
      #print('AUC for Class {}: {}'.format(i+1, auc(fpr[i], tpr[i])))
      auc_avg += auc(fpr[i], tpr[i])
      counting = i+1
    return auc_avg/counting
#---------------------------------------------------------------------
# Defining features of interest
print('---------------------------------------------------------------------------------')
print('Defining features of interest')
print('---------------------------------------------------------------------------------')
print('')

req_cols = ['FLOW_DURATION_MILLISECONDS','FIRST_SWITCHED',
            'TOTAL_FLOWS_EXP','TCP_WIN_MSS_IN','LAST_SWITCHED',
            'TCP_WIN_MAX_IN','TCP_WIN_MIN_IN','TCP_WIN_MIN_OUT',
           'PROTOCOL','TCP_WIN_MAX_OUT','TCP_FLAGS',
            'TCP_WIN_SCALE_OUT','TCP_WIN_SCALE_IN','SRC_TOS',
            'DST_TOS','FLOW_ID','L4_SRC_PORT','L4_DST_PORT',
           'MIN_IP_PKT_LEN','MAX_IP_PKT_LEN','TOTAL_PKTS_EXP',
           'TOTAL_BYTES_EXP','IN_BYTES','IN_PKTS','OUT_BYTES','OUT_PKTS',
            'ALERT']
#---------------------------------------------------------------------
#Load Databases from csv file
address = '/home/oarreche@ads.iu.edu/HITL/sensor/sensor_db'
print('Loading Database')
print('--------------------------------------------------')

fraction = 0.1
fraction2 = 0.01

#Denial of Service
df0 = pd.read_csv (address + '/dos-03-15-2022-15-44-32.csv', usecols=req_cols).sample(frac = fraction)
df1 = pd.read_csv (address + '/dos-03-16-2022-13-45-18.csv', usecols=req_cols).sample(frac = fraction)
df2 = pd.read_csv (address + '/dos-03-17-2022-16-22-53.csv', usecols=req_cols).sample(frac = fraction)
df3 = pd.read_csv (address + '/dos-03-18-2022-19-27-05.csv', usecols=req_cols).sample(frac = fraction)
df4 = pd.read_csv (address + '/dos-03-19-2022-20-01-53.csv', usecols=req_cols).sample(frac = fraction)
df5 = pd.read_csv (address + '/dos-03-20-2022-14-27-54.csv', usecols=req_cols).sample(frac = fraction)


#Malware
#df6 = pd.read_csv ('sensor_db/malware-03-25-2022-17-57-07.csv', usecols=req_cols)

#Normal
df7 = pd.read_csv  (address + '/normal-03-15-2022-15-43-44.csv', usecols=req_cols).sample(frac = fraction2)
df8 = pd.read_csv  (address + '/normal-03-16-2022-13-44-27.csv', usecols=req_cols).sample(frac = fraction2)
df9 = pd.read_csv  (address + '/normal-03-17-2022-16-21-30.csv', usecols=req_cols).sample(frac = fraction2)
df10 = pd.read_csv (address + '/normal-03-18-2022-19-17-31.csv', usecols=req_cols).sample(frac = fraction2)
df11 = pd.read_csv (address + '/normal-03-18-2022-19-25-48.csv', usecols=req_cols).sample(frac = fraction2)
df12 = pd.read_csv (address + '/normal-03-19-2022-20-01-16.csv', usecols=req_cols).sample(frac = fraction2)
df13 = pd.read_csv (address + '/normal-03-20-2022-14-27-30.csv', usecols=req_cols).sample(frac = fraction2)


#PortScanning
df14 = pd.read_csv  (address + '/portscanning-03-15-2022-15-44-06.csv', usecols=req_cols).sample(frac = fraction)
df15 = pd.read_csv  (address + '/portscanning-03-16-2022-13-44-50.csv', usecols=req_cols).sample(frac = fraction)
df16 = pd.read_csv  (address + '/portscanning-03-17-2022-16-22-53.csv', usecols=req_cols).sample(frac = fraction)
df17 = pd.read_csv  (address + '/portscanning-03-18-2022-19-27-05.csv', usecols=req_cols).sample(frac = fraction)
df18 = pd.read_csv  (address + '/portscanning-03-19-2022-20-01-45.csv', usecols=req_cols).sample(frac = fraction)
df19 = pd.read_csv  (address + '/portscanning-03-20-2022-14-27-49.csv', usecols=req_cols).sample(frac = fraction)


frames = [df0, df1, df2, df3, df4, df5, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

# fraction = 0.1

#concat data frames
df = pd.concat(frames,ignore_index=True)

# shuffle the DataFrame rows
# df = df.sample(frac = fraction)

y = df.pop('ALERT')
X = df

df_max_scaled = X
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)



# IG

# df.pop('TOTAL_FLOWS_EXP')
# df.pop('LAST_SWITCHED')
# df.pop('TCP_WIN_MAX_IN')
# df.pop('TCP_WIN_MIN_IN')
# df.pop('FIRST_SWITCHED')

# df.pop('L4_SRC_PORT')
# df.pop('TCP_WIN_SCALE_IN')
# df.pop('FLOW_ID')
# df.pop('L4_DST_PORT')
# df.pop('PROTOCOL')

# df.pop('TCP_WIN_MSS_IN')
# df.pop('TCP_WIN_MIN_OUT')
# df.pop('DST_TOS')
# df.pop('FLOW_DURATION_MILLISECONDS')
# df.pop('TCP_FLAGS')
# df.pop('TCP_WIN_MAX_OUT')
# df.pop('TCP_WIN_SCALE_OUT')
# df.pop('SRC_TOS')
# df.pop('IN_PKTS')
# df.pop('OUT_PKTS')
# df.pop('OUT_BYTES')

# df.pop('IN_BYTES')
# df.pop('MIN_IP_PKT_LEN')
# df.pop('MAX_IP_PKT_LEN')
# df.pop('TOTAL_PKTS_EXP')
# df.pop('TOTAL_BYTES_EXP')


# LRP 
# df.pop('TOTAL_FLOWS_EXP')
# df.pop('LAST_SWITCHED')
# df.pop('TCP_WIN_MIN_IN')
# df.pop('TCP_WIN_MAX_IN')
# df.pop('FIRST_SWITCHED')

# df.pop('TCP_WIN_SCALE_IN')
# df.pop('L4_SRC_PORT')
# df.pop('L4_DST_PORT')
# df.pop('FLOW_ID')
# df.pop('PROTOCOL')

# df.pop('TCP_WIN_MSS_IN')
# df.pop('TCP_FLAGS')
# df.pop('FLOW_DURATION_MILLISECONDS')
# df.pop('DST_TOS')
# df.pop('TCP_WIN_MIN_OUT')
# df.pop('TCP_WIN_MAX_OUT')
# df.pop('TCP_WIN_SCALE_OUT')
# df.pop('SRC_TOS')
# df.pop('IN_PKTS')
# df.pop('OUT_PKTS')
# df.pop('OUT_BYTES')

# df.pop('IN_BYTES')
# df.pop('MIN_IP_PKT_LEN')
# df.pop('MAX_IP_PKT_LEN')
# df.pop('TOTAL_PKTS_EXP')
# df.pop('TOTAL_BYTES_EXP')


# SHAP
# df.pop('TCP_WIN_MIN_IN')
# df.pop('TCP_WIN_MIN_OUT')
# df.pop('TCP_WIN_SCALE_OUT')
# df.pop('FLOW_DURATION_MILLISECONDS')
# df.pop('TCP_WIN_MAX_IN')

# df.pop('TCP_WIN_SCALE_IN')
# df.pop('TCP_WIN_MAX_OUT')
# df.pop('FLOW_ID')
# df.pop('L4_SRC_PORT')
# df.pop('TOTAL_FLOWS_EXP')

# df.pop('TCP_FLAGS')
# df.pop('TCP_WIN_MSS_IN')
# df.pop('LAST_SWITCHED')
# df.pop('FIRST_SWITCHED')
# df.pop('IN_PKTS')
# df.pop('OUT_PKTS')
# df.pop('IN_BYTES')
# df.pop('OUT_BYTES')
# df.pop('L4_DST_PORT')
# df.pop('SRC_TOS')
# df.pop('DST_TOS')

# df.pop('PROTOCOL')
# df.pop('MIN_IP_PKT_LEN')
# df.pop('MAX_IP_PKT_LEN')
# df.pop('TOTAL_PKTS_EXP')
# df.pop('TOTAL_BYTES_EXP')
#---------------------------------------------------------------------

# Separate features and labels 
print('---------------------------------------------------------------------------------')
print('Separating features and labels')
print('---------------------------------------------------------------------------------')
print('')

y = df.pop('Label')
X = df
# summarize class distribution
counter = Counter(y)
print(counter)
# transform the dataset
print('---------------------------------------------------------------------------------')
result_list = [counter['None'],counter['Denial of Service'], counter['Port Scanning']]
print('number of Labels  ',result_list)
print('---------------------------------------------------------------------------------')

df = X.assign( Label = y)

#---------------------------------------------------------------------

# Separate Training and Testing db
print('---------------------------------------------------------------------------------')
print('Separating Training and Testing db')
print('---------------------------------------------------------------------------------')
print('')

# y = df_train.pop('ALERT')
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .70
#print(df.head())

train, test = df[df['is_train']==True], df[df['is_train']==False]
print('Number of the training data:', len(train))
print('Number of the testing data:', len(test))

features = df.columns[:len(req_cols)-1]

y_train, label = pd.factorize(train['Label'])
y_test, label = pd.factorize(test['Label'])

#---------------------------------------------------------------------
# Defining the DNN model

df_y, df_label = pd.factorize(df['Label'])


df2 = df.drop(columns=['Label','is_train'])
df2['Label'] = df_y
df2
train = train.drop(columns=['Label','is_train'])
# # uncomented for Information Gain
# from sklearn.feature_selection import mutual_info_classif
# %matplotlib inline

# # Compute information gain using mutual information
# importances0 = mutual_info_classif(train, y_train)


# feat_importances0 = pd.Series(importances0, df2.columns[0:len(df2.columns)-1])

    
# feat_importances_sorted0 = feat_importances0.sort_values( ascending=False)


# # Print or use the sorted DataFrame
# print(feat_importances_sorted0)

# # feat_importances_sorted.plot(kind='barh', color = 'teal')
# # feat_importances_sorted
# top_features0 = feat_importances_sorted0.nlargest(10)

# top_feature_names0 = top_features0.index.tolist()


# print("Top 5 feature names:")
# print(top_features0)

# column_features = top_feature_names0

# top10 = ['IN_BYTES'
#          ,'FLOW_ID'
#          ,'TOTAL_FLOWS_EXP'
#          ,'LAST_SWITCHED'
#          ,'FIRST_SWITCHED'
#          ,'TCP_WIN_MAX_IN'
#          ,'TCP_WIN_MIN_IN'
#          ,'TCP_FLAGS'
#          ,'FLOW_DURATION_MILLISECONDS'
#          ,'L4_SRC_PORT']


test.pop('is_train')
test.pop('Label')
y_test
y_train

X_train = train
X_test = test


---------------------------------------------------------------------------------
Initializing DNN program
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Importing Libraries
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Defining Metric Equations
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Defining features of interest
---------------------------------------------------------------------------------

Loading Database
--------------------------------------------------
---------------------------------------------------------------------------------
Separating features and labels
----------------------------------------------------------

In [32]:
import numpy as np

import tensorflow as tf

# Now you can use Keras modules directly from tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Flatten
import innvestigate
tf.compat.v1.disable_eager_execution()

In [33]:
from tensorflow.keras.callbacks import LearningRateScheduler

print('---------------------------------------------------------------------------------')
print('Defining the DNN model')
print('---------------------------------------------------------------------------------')
print('')

# Define the number of nodes per layer
nodes_first_layer = 128
nodes_second_layer = 64
nodes_third_layer = 32

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(X_train.columns,))))

# First dense layer
model.add(tf.keras.layers.Dense(nodes_first_layer, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))  # Dropout layer follows the first dense layer

# Second dense layer
model.add(tf.keras.layers.Dense(nodes_second_layer, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))  # Dropout layer follows the second dense layer

# Third dense layer
model.add(tf.keras.layers.Dense(nodes_third_layer, activation='relu'))

# Output layer, assuming the task involves classification into 3 classes
model.add(tf.keras.layers.Dense(3))  # No activation here, add 'softmax' when using the model for prediction






# model.add(tf.keras.layers.Dense(3))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


#---------------------------------------------------------------------

# # Define your DNN model
# model = Sequential([
#     Flatten(input_shape=(28, 28)),  # Input layer
#     Dense(128, activation='relu'),  # Hidden layer
#     Dense(10, activation='relu')  # Output layer
# ])

# # Compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])



# # Save the trained model
# model.save('your_model.h5')

# # Load your trained model
# model = tf.keras.models.load_model('your_model.h5')


---------------------------------------------------------------------------------
Defining the DNN model
---------------------------------------------------------------------------------



In [34]:
from tensorflow.keras.models import load_model

from tensorflow.keras.callbacks import EarlyStopping
# Define your learning rate schedule function
# def lr_schedule(epoch):
#     # Your learning rate schedule logic here
#     learning_rate = 0.1
#     if epoch > 10:
#         learning_rate = 0.01
#     if epoch > 20:
#         learning_rate = 0.001
#     return learning_rate
# lr_sched = LearningRateScheduler(lambda epoch: 1e-3 * (0.75 ** np.floor(epoch / 2)))

# # Create a LearningRateScheduler callback
# lr_scheduler = LearningRateScheduler(lr_schedule)
print('---------------------------------------------------------------------------------')
print('Training the model')
print('---------------------------------------------------------------------------------')
print('')


# Define a learning rate scheduler
def lr_schedule(epoch, lr):
    if epoch > 10:
        return lr * 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

start = time.time()

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)

# Modify model.fit to include the EarlyStopping callback
model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping,lr_scheduler])

# Pass the lr_scheduler callback to your model.fit() function
# model.fit(X_train, y_train, callbacks=[lr_scheduler], ...)


end = time.time()
print('---------------------------------------------------------------------------------')
print('ELAPSE TIME TRAINING MODEL: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')


---------------------------------------------------------------------------------
Training the model
---------------------------------------------------------------------------------



Train on 878476 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 1/1000
878476/878476 [==============================] - 1s 2us/sample - loss: 4.2339 - accuracy: 0.5103 - lr: 0.0010

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 2/1000
878476/878476 [==============================] - 1s 1us/sample - loss: 3.3813 - accuracy: 0.5094 - lr: 0.0010

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 3/1000
878476/878476 [==============================] - 1s 1us/sample - loss: 2.9274 - accuracy: 0.5093 - lr: 0.0010

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 4/1000
878476/878476 [==============================] - 1s 1us/sample - loss: 2.7118 - accuracy: 0.5102 - lr: 0.0010

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 5/1000
878476/878476 [==============================]

In [35]:

print('---------------------------------------------------------------------------------')
print('Model Prediction')
print('---------------------------------------------------------------------------------')
print('')
print('---------------------------------------------------------------------------------')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('ELAPSE TIME MODEL PREDICTION: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')

#print(y_pred)
ynew = np.argmax(y_pred,axis = 1)
#print(ynew)
score = model.evaluate(X_test, y_test,verbose=1)
#print(score)
pred_label = label[ynew]
#print(score)

#---------------------------------------------------------------------
# pd.crosstab(test['ALERT'], preds, rownames=['Actual ALERT'], colnames = ['Predicted ALERT'])

from sklearn.metrics import accuracy_score

accuracy =accuracy_score(y_test, ynew)*100
print(accuracy)

from collections import Counter

label_counts = Counter(y_test)
print(label_counts)

label_counts = Counter(ynew)
print(label_counts)

accuracy =accuracy_score(y_test, ynew)*100
print(accuracy)

---------------------------------------------------------------------------------
Model Prediction
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


ELAPSE TIME MODEL PREDICTION:  0.16981900533040364 min
---------------------------------------------------------------------------------

92.39697291556027
Counter({0: 192329, 2: 125302, 1: 58969})
Counter({0: 200863, 2: 130289, 1: 45448})
92.39697291556027


In [36]:
single_sample_df = X_test.iloc[[0]]

## Integrated Gradients


In [48]:
%%time
# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Replace X_test with your input data

# X_test = np.random.rand(100, 28, 28)  # Example input data

# X_test2 = X_test.frac(0.001, random_state=42)

# Perform LRP analysis on the input data
analysis = analyzer.analyze(X_test)
#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)

# Perform LRP analysis on a certain number of samples
analysis = analyzer.analyze(X_test.sample(100))

# Print or use the analysis results as needed
print(analysis)

print(len(X_test))
print(len(X_test.columns))
names = X_test.columns
print(analysis.shape)
print(type(analysis))
scores = pd.DataFrame(analysis)
print(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

names

sum_of_columns = list(sum_of_columns)

sum_of_columns
# type(sum_of_columns)
# # sorted_series = sum_of_columns.sort_values(ascending=False)

# print("Sorted Series in descending order:")
# print(sorted_series)
### Results
# names = ['John', 'Alice', 'Bob', 'Emily']
# sum_of_columns = [10, 5, 15, 8]

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)


sorted_names


[[ 0.129  0.028  0.    ...  0.     0.     0.   ]
 [-0.014  0.076  0.016 ...  0.    -0.    -0.   ]
 [-0.01   0.073  0.    ...  0.     0.     0.   ]
 ...
 [ 0.107  0.032  0.    ...  0.     0.     0.   ]
 [ 0.129  0.02   0.    ...  0.     0.     0.   ]
 [ 0.124  0.026  0.    ...  0.     0.     0.   ]]
376600
26
(100, 26)
<class 'numpy.ndarray'>
[[ 0.129  0.028  0.    ...  0.     0.     0.   ]
 [-0.014  0.076  0.016 ...  0.    -0.    -0.   ]
 [-0.01   0.073  0.    ...  0.     0.     0.   ]
 ...
 [ 0.107  0.032  0.    ...  0.     0.     0.   ]
 [ 0.129  0.02   0.    ...  0.     0.     0.   ]
 [ 0.124  0.026  0.    ...  0.     0.     0.   ]]
('FLOW_ID', 'TCP_WIN_MAX_IN', 'TCP_WIN_MIN_IN', 'TOTAL_FLOWS_EXP', 'L4_SRC_PORT', 'FIRST_SWITCHED', 'LAST_SWITCHED', 'TCP_WIN_SCALE_IN', 'L4_DST_PORT', 'FLOW_DURATION_MILLISECONDS', 'PROTOCOL', 'TCP_FLAGS', 'TCP_WIN_MSS_IN', 'DST_TOS', 'SRC_TOS', 'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_OUT', 'IN_PKTS', 'OUT_PKTS', 'IN_BYTES', 'OUT_BYTES', 'TCP_WIN_SCALE_OUT', 'M

('FLOW_ID',
 'TCP_WIN_MAX_IN',
 'TCP_WIN_MIN_IN',
 'TOTAL_FLOWS_EXP',
 'L4_SRC_PORT',
 'FIRST_SWITCHED',
 'LAST_SWITCHED',
 'TCP_WIN_SCALE_IN',
 'L4_DST_PORT',
 'FLOW_DURATION_MILLISECONDS',
 'PROTOCOL',
 'TCP_FLAGS',
 'TCP_WIN_MSS_IN',
 'DST_TOS',
 'SRC_TOS',
 'TCP_WIN_MAX_OUT',
 'TCP_WIN_MIN_OUT',
 'IN_PKTS',
 'OUT_PKTS',
 'IN_BYTES',
 'OUT_BYTES',
 'TCP_WIN_SCALE_OUT',
 'MIN_IP_PKT_LEN',
 'MAX_IP_PKT_LEN',
 'TOTAL_PKTS_EXP',
 'TOTAL_BYTES_EXP')

## LRP


In [45]:
%%time

# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("lrp.z", model)

# Replace X_test with your input data

# X_test = np.random.rand(100, 28, 28)  # Example input data

# X_test2 = X_test.frac(0.001, random_state=42)

# Perform LRP analysis on the input data
analysis = analyzer.analyze(X_test)

#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(1))


# Print or use the analysis results as needed
print(analysis)
print(len(X_test))
print(len(X_test.columns))
names = X_test.columns
print(analysis.shape)
print(type(analysis))
scores = pd.DataFrame(analysis)
print(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

names

sum_of_columns = list(sum_of_columns)

sum_of_columns
# type(sum_of_columns)
# # sorted_series = sum_of_columns.sort_values(ascending=False)

# print("Sorted Series in descending order:")
# print(sorted_series)
# names = ['John', 'Alice', 'Bob', 'Emily']
# sum_of_columns = [10, 5, 15, 8]

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names


[[-0.001  0.056  0.036  0.028 -0.     0.05   0.002 -0.001  0.001 -0.
   0.     0.     0.004 -0.     0.    -0.    -0.    -0.026 -0.    -0.
   0.    -0.    -0.     0.    -0.    -0.   ]]
376600
26
(1, 26)
<class 'numpy.ndarray'>
[[-0.001  0.056  0.036  0.028 -0.     0.05   0.002 -0.001  0.001 -0.
   0.     0.     0.004 -0.     0.    -0.    -0.    -0.026 -0.    -0.
   0.    -0.    -0.     0.    -0.    -0.   ]]
('L4_SRC_PORT', 'LAST_SWITCHED', 'L4_DST_PORT', 'FIRST_SWITCHED', 'TOTAL_FLOWS_EXP', 'TCP_WIN_MSS_IN', 'PROTOCOL', 'TCP_FLAGS', 'FLOW_ID', 'TCP_WIN_MAX_IN', 'TCP_WIN_MIN_IN', 'IN_PKTS', 'OUT_PKTS', 'IN_BYTES', 'OUT_BYTES', 'FLOW_DURATION_MILLISECONDS', 'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_OUT', 'TCP_WIN_SCALE_IN', 'TCP_WIN_SCALE_OUT', 'SRC_TOS', 'DST_TOS', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN', 'TOTAL_PKTS_EXP', 'TOTAL_BYTES_EXP')
(0.05625816807150841, 0.049711477011442184, 0.03615616261959076, 0.028453459963202477, 0.026484090834856033, 0.003746533300727606, 0.001537513337098062, 0.00140938

('L4_SRC_PORT',
 'LAST_SWITCHED',
 'L4_DST_PORT',
 'FIRST_SWITCHED',
 'TOTAL_FLOWS_EXP',
 'TCP_WIN_MSS_IN',
 'PROTOCOL',
 'TCP_FLAGS',
 'FLOW_ID',
 'TCP_WIN_MAX_IN',
 'TCP_WIN_MIN_IN',
 'IN_PKTS',
 'OUT_PKTS',
 'IN_BYTES',
 'OUT_BYTES',
 'FLOW_DURATION_MILLISECONDS',
 'TCP_WIN_MAX_OUT',
 'TCP_WIN_MIN_OUT',
 'TCP_WIN_SCALE_IN',
 'TCP_WIN_SCALE_OUT',
 'SRC_TOS',
 'DST_TOS',
 'MIN_IP_PKT_LEN',
 'MAX_IP_PKT_LEN',
 'TOTAL_PKTS_EXP',
 'TOTAL_BYTES_EXP')

## SHAP


In [42]:
%%time
output_file_name = 'SHAPCIC.txt'
with open(output_file_name, "w") as f:print('',file = f)

###here

print('---------------------------------------------------------------------------------')
print('Generating Explainer')
print('---------------------------------------------------------------------------------')

samples = 1

#uncomment for single sample
# samples = 1

Label = label
# df.pop('Label')
#train.pop('Label')
test = X_test
train = X_train
#df.pop('is_train')
start_index = 0

#end_index = len(test)
end_index = samples
#test = test[features]
explainer = shap.DeepExplainer(model,train[start_index:end_index].values.astype('float'))
shap_values = explainer.shap_values(test[start_index:end_index].values.astype('float'))
# shap_values = explainer.shap_values(test[start_index:len(test)].values.astype('float'))

# shap.summary_plot(shap_values = shap_values,
#                  features = test[start_index:end_index],
#                   class_names=[label[0],label[1],label[2],label[3],label[4],label[5],label[6]],show=False)

# plt.savefig('DNN_Shap_Summary_Cicids.png')
# plt.clf()

vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()


# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)


# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

for k in sorted_list1:
  with open(output_file_name, "a") as f:print("df.pop('",k,"')", sep='',file = f)
with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')




# # shap.summary_plot(shap_values = shap_values[0],
# #                  features = test[start_index:end_index],
# #                   class_names=[label[0],label[1],label[2],label[3],label[4],label[5],label[6]],show=False)


# # plt.savefig('DNN_Shap_Summary_Beeswarms.png')
# # plt.clf()



# print('---------------------------------------------------------------------------------')
# print('Generating Sparsity Graph')
# print('---------------------------------------------------------------------------------')
# print('')

# # Find the minimum and maximum values in the list
# min_value = min(feature_val)
# max_value = max(feature_val)

# # Normalize the list to the range [0, 1]
# normalized_list = []
# for x in feature_val:
#     if max_value - min_value == 0:
#         normalized_list.append(0)
#     else:
#         normalized_list.append((x - min_value) / (max_value - min_value))
   
# # print(feature_name,normalized_list,'\n')
# # for item1, item2 in zip(feature_name, normalized_list):
# #     print(item1, item2)

# #calculating Sparsity

# # Define the threshold
# threshold = 1e-10

# # Initialize a count variable to keep track of values below the threshold
# count_below_threshold = 0

# # Iterate through the list and count values below the threshold
# for value in normalized_list:
#     if value < threshold:
#         count_below_threshold += 1

# Sparsity = count_below_threshold/len(normalized_list)
# Spar = []
# print('Sparsity = ',Sparsity)
# X_axis = []
# #----------------------------------------------------------------------------
# for i in range(0, 11):
#     i/10
#     threshold = i/10
#     for value in normalized_list:
#         if value < threshold:
#             count_below_threshold += 1

#     Sparsity = count_below_threshold/len(normalized_list)
#     Spar.append(Sparsity)
#     X_axis.append(i/10)
#     count_below_threshold = 0


# #---------------------------------------------------------------------------

# with open(output_file_name, "a") as f:print('y_axis_RF = ', Spar ,'', file = f)
# with open(output_file_name, "a") as f:print('x_axis_RF = ', X_axis ,'', file = f)

# plt.clf()

# # Create a plot
# plt.plot(X_axis, Spar, marker='o', linestyle='-')

# # Set labels for the axes
# plt.xlabel('X-Axis')
# plt.ylabel('Y-Axis')

# # Set the title of the plot
# plt.title('Values vs. X-Axis')

# # Show the plot
# # plt.show()
# plt.savefig('sparsity.png')
# plt.clf()

---------------------------------------------------------------------------------
Generating Explainer
---------------------------------------------------------------------------------


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


                      col_name  feature_importance_vals
1                  L4_SRC_PORT             4.263129e-02
0                      FLOW_ID             4.591741e-07
4   FLOW_DURATION_MILLISECONDS             4.117334e-07
17             TOTAL_FLOWS_EXP             3.589247e-07
5                LAST_SWITCHED             4.495109e-10
3               FIRST_SWITCHED             3.042478e-10
15                     SRC_TOS             0.000000e+00
24                   OUT_BYTES             0.000000e+00
23                     IN_PKTS             0.000000e+00
22                    IN_BYTES             0.000000e+00
21             TOTAL_BYTES_EXP             0.000000e+00
20              TOTAL_PKTS_EXP             0.000000e+00
19              MAX_IP_PKT_LEN             0.000000e+00
18              MIN_IP_PKT_LEN             0.000000e+00
16                     DST_TOS             0.000000e+00
13            TCP_WIN_SCALE_IN             0.000000e+00
14           TCP_WIN_SCALE_OUT             0.000